# S&P 500 Analysis

Finding trends and relationships among S&P 500 stocks using as much information as possible, including time series, trends, financial ratios, macroeconomic data, headlines, analyst and general sentiment (from Reddit, for example), SEC filings, etc. Data is collected using a variety of methods, APIs when possible and web scraping when necessary.

More companies from outside the S&P 500 will be added over time, as will derivatives. The end goal is to try to construct an algorithm that can beat the S&P 500 in the long run, with as much automation as possible.

Any information, trends, and models that could be valuable will, of course, not be here.

## Data Collection

Current data and sources:

Stock prices: Quandl<br>
Headlines and Summaries: Yahoo Finance<br>
Financial Statements: Yahoo Finance

In [9]:
# Scrape Yahoo finance for ALL data: finances, prices, sector name, options, headlines, product categories, etc.
# Eventually scrape more: twitter headlines, google trends, etc.
# Start with ABT - Abbott Laboratories, then generalize

from bs4 import BeautifulSoup # For HTML parsing
import urllib # Website connections
import re # Regular expressions
from time import sleep # To prevent overwhelming the server between connections
#from collections import Counter # Keep track of our term counts
#from nltk.corpus import stopwords # Filter out stopwords, such as 'the', 'or', 'and'
import pandas as pd # For converting results to a dataframe and bar chart plots
import numpy as np
import datetime
import json
import sys
import time
import quandl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import os
quandl.ApiConfig.api_key = "INSERT API KEY HERE"
%matplotlib inline
today = datetime.datetime.now()

In [10]:
# First scrape info on S&P 500 companies from https://en.wikipedia.org/wiki/List_of_S%26P_500_companies
# Get the name, ticker.
# Can also be done simply by using Pandas to read the table.

def get_wiki_info():
    site = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    html = urllib.request.urlopen(site).read()
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table',{'class':'wikitable sortable'})
    rows = table.findAll('tr')
    
    labels = ['name','filings','gics','gics_sub','hq','added','cik']
    
    collector = {}
    for r in range(1,len(rows)):
        columns = rows[r].findAll('td')
        ticker = columns[0].string
        name = columns[1].string
        filings = columns[2].findAll('a')[0].get('href')
        gics = columns[3].string
        gics_sub = columns[4].string
        hq = columns[5].string
        try:
            added = datetime.datetime.strptime(columns[6].string, '%Y-%m-%d')
        except:
            added = None
        cik = columns[7].string
        collector[ticker] = [name,filings,gics,gics_sub,hq,added,cik]
    
    collector[ticker] = [name,filings,gics,gics_sub,hq,added,cik]
    df = pd.DataFrame(collector)
    df.index = labels
    df.to_json('companies.json')
    return df

In [11]:
def get_sp_tickers():
    with open('companies.json') as company_data:
        data = json.load(company_data)
    
    tickers = [key for key,value in data.items()]
    return tickers

In [12]:
# Get financial info. Does not do analysis, which will be done in another function.
# Would be better to have a higher level scrape funtion call the specific financials, history, headlines, etc. functions.

def get_financials(ticker):
    url = 'https://finance.yahoo.com/quote/'+ticker+'/financials?p='+ticker
    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html,'html.parser')

    soup_script = soup.find("script",text=re.compile("root.App.main")).text
    json_script = json.loads(re.search("root.App.main\s+=\s+(\{.*\})",soup_script)[1])
    fin_info = json_script['context']['dispatcher']['stores']['QuoteSummaryStore']

    # First get the balance sheets
    bal_yr = fin_info['balanceSheetHistory']['balanceSheetStatements']
    bal_qtrs = fin_info['balanceSheetHistoryQuarterly']['balanceSheetStatements']
    
    # Keep only the raw values in the balance sheet
    bal_yr = {'yearly_balance':get_raw_numbers(bal_yr)}
    bal_qtrs = {'quarterly_balance':get_raw_numbers(bal_qtrs)}

    # Now get the income statements
    inc_yr = fin_info['incomeStatementHistory']['incomeStatementHistory']
    inc_qtrs = fin_info['incomeStatementHistoryQuarterly']['incomeStatementHistory']
    inc_yr = {'yearly_income:':get_raw_numbers(inc_yr)}
    inc_qtrs = {'quarterly_income':get_raw_numbers(inc_qtrs)}
    
    # Cash flow
    cash_yr = fin_info['cashflowStatementHistory']['cashflowStatements']
    cash_qtrs = fin_info['cashflowStatementHistoryQuarterly']['cashflowStatements']
    cash_yr = {'yearly_cash':get_raw_numbers(cash_yr)}
    cash_qtrs = {'quarterly_cash':get_raw_numbers(cash_qtrs)}
    
    combined_dict = {'financials':{**bal_yr,**bal_qtrs, **inc_yr, **bal_qtrs, **cash_yr, **cash_qtrs}}
    
    return combined_dict
        
def get_raw_numbers(fin_dict):
    for i in range (len(fin_dict)):
        fin_dict[i].pop('maxAge')                # this is just 1 for everything, not sure what it does
        for key,value in fin_dict[i].items():
            try:
                fin_dict[i][key] = value['raw']
            except:                              # a few have only the raw value for some reason
                fin_dict[i][key] = value
    
    year_keys = [datetime.datetime.fromtimestamp(int(fin_dict[i]['endDate'])).strftime('%Y-%m-%d') for i in range(len(fin_dict))]
    return dict(zip(year_keys,fin_dict))

In [13]:
# Get all financial statements from all companies in S&P 500
# Yearly - 3 most recent, Quarterly - 4 most recent
def get_all_financials():
    tickers = get_sp_tickers()   
    
    all_fin = {}
    # Can fetch quandl data at the same time since they won't overlap
    for ticker in tickers:
        print ('Fetching ', ticker)
        all_fin[ticker] = get_financials(ticker)
        sleep(1)
    
    with open('allfinancial.json', 'w') as outfile:
        json.dump(all_fin, outfile)

In [14]:
def get_stock_hist(start, end):
    with open('companies.json') as company_data:
        data = json.load(company_data)
    tickers = [key.replace('.','_') for key,value in data.items()]
    
    all_hist = pd.DataFrame()
    for ticker in tickers:
        print ('Fetching ',ticker)        
        newdata = pd.DataFrame(quandl.get('WIKI/'+ticker, start_date=start, end_date=end)).add_prefix(ticker.replace('_','.')+' ')
        all_hist = pd.concat((all_hist,newdata),axis=1)
    
    df = pd.DataFrame(all_hist)
    df.to_csv('stockhist_'+start+'_'+end+'.csv')
    return all_hist

In [86]:
# Filter just the adjusted closing prices
def get_hist_closing (hist):
    tickers = get_sp_tickers()
    close_prices = pd.concat([hist[col] for col in hist if 'Adj. Close' in col],axis=1)
    close_prices.columns = tickers
    return close_prices
    
# Create a correlation matrix (use with adjusted close or open for consistency)    
def get_corr_mat(l,p,data):
    starting_loc = -1-(l*p)
    corr_sum = data.iloc[starting_loc:starting_loc+l].corr()
    for i in range(1, p):
        corr_sum += data.iloc[starting_loc+i*l:starting_loc+(i+1)*l].corr()
    data = corr_sum/p
    return data

# Get the daily log(current/previous) approximation,
def get_hist_returns (hist):
    tickers = get_sp_tickers()
    close_prices = get_hist_closing (hist)

    close_returns = pd.DataFrame()
    for i,t in enumerate(close_prices):
        close_returns[t] = close_prices[t]/close_prices[t].shift(1)-1           # alternatively, use close_prices[t].pct_change()
    
    return close_returns
    
def get_cov_mat (l,p,data):
    starting_loc = -1-(l*p)
    corr_sum = data.iloc[starting_loc:starting_loc+l].corr()
    for i in range(1, p):
        corr_sum += data.iloc[starting_loc+i*l:starting_loc+(i+1)*l].corr()
    data = corr_sum/p
    return data

In [16]:
# Get financial info. Each company has different entries, but there is enough in common for most standard ratios.
# Using different scraping packages such as Selenium, it would be possible to gather even more information from each page
# by getting the headlines below the javascript scroll.

def get_recent_headlines(ticker):
    url = 'https://finance.yahoo.com/quote/'+ticker+'/news?p='+ticker
    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html,'html.parser')

    soup_script = soup.find("script",text=re.compile("root.App.main")).text
    json_script = json.loads(re.search("root.App.main\s+=\s+(\{.*\})",soup_script)[1])
    all_heads = json_script['context']['dispatcher']['stores']['StreamStore']['streams']['YFINANCE:'+ticker+'.mega']['data']['stream_items']
    
    all_heads = process_headlines (all_heads)
    
    return all_heads

def process_headlines(heads):
    # We need to account for editor's picks articles which have a different forma
    reorg = {}
    for head in heads:
        for e in ['clusterInfo','editorsPick','format','storyline','i13n','id',
                  'idPrefix','images','is_eligible','link','off_network','type']:
            try:
                head.pop(e)
            except:
                pass
        head['pubtime'] = datetime.datetime.fromtimestamp(int(head['pubtime'])//1000).strftime('%Y-%m-%d')
        if 'summary' not in head:
            head['summary'] = '(NO SUMMARY)'
        
        if head['pubtime'] in reorg:
            reorg[head['pubtime']][head['title']] = {'summary':head['summary'],
                                                   'publisher': head['publisher'],
                                                   'url': head['url']}
        else:
            reorg[head['pubtime']]={head['title']:{'summary':head['summary'],
                                                   'publisher': head['publisher'],
                                                   'url': head['url']}}
    
    # Store the number of headlines each day, as a large number of headlines might have a correlation
    # with price movement in either direction.
    for day in reorg:
        daily_heads = len(reorg[day])
        reorg[day]['daily_headlines'] = daily_heads
        
    return reorg

# Run this to get a time-labeled json file
def get_all_headlines ():
    tickers = get_sp_tickers()   
    all_heads = {}
    
    for ticker in tickers:
        print ('Fetching ', ticker)
        try:
            all_heads[ticker] = get_recent_headlines(ticker)
        except:
            # Need to try a few times
            counter = 1
            while counter <= 5:
                sleep(10)
                all_heads[ticker] = get_recent_headlines(ticker)
                counter += 1
        sleep(2)
    
    with open('allheadlines_'+datetime.datetime.now().strftime('%Y-%m-%d')+'.json', 'w') as outfile:
        json.dump(all_heads, outfile)

In [17]:
def merge_ts_csv (input_files = (), *args, output_file):
    collector = pd.DataFrame()
    for file in input_files:
        current_series = pd.read_csv(file).set_index('Date')
        collector = collector.combine_first(current_series)
    collector.to_csv(output_file)

In [18]:
def merge_json_headlines (input_files = (), *args, output_file):
    head_list = []
    for i in range(len(input_files)):
        with open (input_files[i]) as current_file:
            head_list.append(json.load(current_file))
    
    collector = head_list[0]
    for i in range(1, len(input_files)):
        for k_time,v_time in head_list[i].items():
            if k_time in collector:
                for k_head, v_head in v_time.items():
                    collector[k_time][k_head] = head_list[i][k_time][k_head]
            else:
                collector[k_time] = head_list[i][k_time]
    
    for ticker in collector:
        for k_time in collector[ticker]:
            daily_heads = len(collector[ticker][k_time])-1
            collector[ticker][k_time]['daily_headlines'] = daily_heads 
    
    with open(output_file, 'w') as outfile:
        json.dump(collector, outfile)
    #return collector

In [20]:
# Gets all current Yahoo Finance headlines for all S&P 500 companies.
#get_all_headlines()

In [22]:
merge_json_headlines(['allheadlines_combined.json','allheadlines_2017-12-16.json','allheadlines_2017-12-08.json','allheadlines_2017-12-13.json','allheadlines_2017-12-18.json'],output_file='allheadlines_combined.json')

In [18]:
# Gets all hlc data for all S&P 500 companies, as well as split/dividend adjusted close data
#get_stock_hist(start = '2017-11-20', end = '2017-12-16')

In [19]:
merge_ts_csv(['stockhist_all.csv','stockhist_2017-11-20_2017-12-16.csv'],output_file='stockhist_all.csv')

In [25]:
with open('companies.json') as company_data:
    meta_data = json.load(company_data)
    
stock_data = pd.read_csv('stockhist_all.csv')

In [34]:
with open('allheadlines_combined.json') as headline_data:
    headline_data = json.load(headline_data)

Headline data is sorted by ticker, then by date.

In [35]:
headline_data['AAPL']

{'2017-11-28': {'Cyber Monday sales hit record highs': {'publisher': 'Yahoo Finance Video',
   'summary': 'Americans shopped until their fingers were sore. Cyber Monday sales hit a new record at around $6.6B, according to Adobe Insights. That’s up 17% compared to last year. Yahoo Finance’s Alexis Christoforous and Dan Roberts breakdown what drove the online buying.',
   'url': 'https://finance.yahoo.com/video/cyber-monday-sales-hit-record-162034065.html'},
  'Here’s why Skype just unfriended Facebook': {'publisher': 'Yahoo Finance Video',
   'summary': "Microsoft says it's killing Facebook login support for Skype. Yahoo Finance’s Seana Smith, Rick Newman, and Justine Underhill debate whether or not this was a smart move.",
   'url': 'https://finance.yahoo.com/video/why-skype-just-unfriended-facebook-182507351.html'},
  'daily_headlines': 2},
 '2017-11-29': {'Nothing about the FAANG stock rally is normal': {'publisher': 'Yahoo Finance',
   'summary': 'Netflix, Facebook and Amazon are ea

### Covariances and Correlations
The wikipedia S&P list includes  different share classes, such as GOOG (non-voting) and GOOGL (voting), for a single company. These shares are purchaseable on the open market, and since they usually move in tandem, they tend to be prime candidates for testing pairs trading algorithms.

Even stocks that move in tandem in the long run may not have strong short-term relationships, especially when there has been a general upward trend in the market. So it would be interesting to look at which companies have strong shorter term correlations.

In [38]:
hist = pd.read_csv('stockhist_all.csv').set_index('Date')

In [50]:
close_prices = get_hist_closing(hist)

In [52]:
close_prices.to_csv('close_prices_'+datetime.datetime.now().strftime('%Y-%m-%d')+'.csv')
close_prices.head()

,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,21.354133,12.394092,39.554636,27.503268,NaN,23.673230,21.229632,35.484403,37.09,25.578015,...,15.983028,20.926528,54.558817,33.821970,28.901835,NaN,21.389761,56.958626,12.807981,NaN
2010-01-05,21.122172,13.797197,39.319541,27.550818,NaN,23.504326,21.058111,35.703703,37.70,25.537633,...,15.872035,20.662679,54.771845,33.419897,28.935325,NaN,21.316612,58.761713,13.259575,NaN
2010-01-06,21.047125,13.225561,39.662387,27.112585,NaN,23.282084,21.175058,36.083260,37.62,25.489174,...,15.709814,20.522510,55.245241,33.640080,28.667405,NaN,21.164220,58.742737,14.412582,NaN
2010-01-07,21.019836,13.615313,39.652592,27.062465,NaN,22.908717,21.350477,36.049522,36.89,25.287264,...,15.709814,20.316377,55.071662,34.080445,28.801365,NaN,21.158125,60.090307,16.026791,NaN
2010-01-08,21.013013,13.355478,39.809321,27.242385,NaN,23.157628,21.459627,35.906134,36.69,25.432639,...,15.539055,20.613208,54.850745,34.080445,28.700895,NaN,21.164220,58.828146,15.767365,NaN


In [85]:
close_returns = get_hist_returns(hist)
close_returns.tail()

,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2017-12-11,-0.005640,0.005488,0.005383,0.019484,0.005419,0.000807,0.005490,0.001063,0.004263,0.003274,...,-0.001356,-0.002334,0.004476,0.001849,0.000000,-0.013345,-0.010442,0.001147,-0.019438,-0.001530
2017-12-12,0.000896,-0.016179,0.021616,-0.005618,-0.001762,0.012101,0.013833,0.009357,-0.010154,-0.003730,...,0.005703,-0.010237,-0.003252,-0.000461,0.005069,0.003864,-0.005579,0.000264,0.010412,0.006409
2017-12-13,-0.005667,-0.007529,-0.005144,0.003320,0.010903,0.007515,-0.011490,-0.004471,0.024864,-0.003510,...,-0.025115,0.005171,0.004350,0.016464,-0.010760,-0.000740,0.002439,-0.000176,-0.013872,0.002077
2017-12-14,-0.002549,0.002396,-0.022439,-0.000290,-0.010786,0.009381,-0.005993,-0.001387,-0.010349,0.003052,...,-0.027424,0.000588,-0.002647,-0.006206,-0.003739,-0.011705,-0.001825,-0.006166,-0.010852,-0.008566
2017-12-15,0.016539,0.016929,0.004990,0.009581,0.011942,0.043220,0.014069,0.009589,0.014229,0.014630,...,0.032469,-0.001322,0.001568,0.012643,0.011600,0.013043,0.003169,0.021450,0.016050,0.002787


In [53]:
# Get five day price correlations for the last 600 days based on log approximations of percentage movements.
fiveday_corr_mat = get_corr_mat(5,120,close_prices)

In [54]:
# Get three day price correlations for the last 600 days
threeday_corr_mat = get_corr_mat(3,200,close_prices)

In [56]:
# Get all the one-to-one correlations for each stock
fiveday_pairwise = fiveday_corr_mat.unstack().reset_index()
threeday_pairwise = threeday_corr_mat.unstack().reset_index()

In [57]:
fiveday_pairwise.columns = ['1','2','corr']
threeday_pairwise.columns = ['1','2','corr']

In [90]:
fiveday_pairwise[(fiveday_pairwise['corr'].abs()>=0.85) & (fiveday_pairwise['corr']<1)].sort_values('corr', ascending=False)[:20]

,1,2,corr
104743,GOOG,GOOGL,0.976890
105247,GOOGL,GOOG,0.976890
69829,DISCA,DISCK,0.956081
70333,DISCK,DISCA,0.956081
98671,FOX,FOXA,0.948367
99175,FOXA,FOX,0.948367
176593,NWSA,NWS,0.937801
176089,NWS,NWSA,0.937801
243062,WEC,ED,0.872056
79766,ED,WEC,0.872056


In [91]:
threeday_pairwise[(threeday_pairwise['corr'].abs()>=0.80) & (threeday_pairwise['corr']<1)].sort_values('corr', ascending=False)[:20]

,1,2,corr
104743,GOOG,GOOGL,0.962917
105247,GOOGL,GOOG,0.962917
98671,FOX,FOXA,0.935794
99175,FOXA,FOX,0.935794
69829,DISCA,DISCK,0.930321
70333,DISCK,DISCA,0.930321
240601,VTR,HCN,0.842275
112081,HCN,VTR,0.842275
145315,LNC,PRU,0.833338
192187,PRU,LNC,0.833338


In [100]:
fiveday_pairwise[(fiveday_pairwise['corr']<-0.20)].sort_values('corr', ascending=True)[:20]

,1,2,corr
243743,WFC,NEM,-0.262413
168647,NEM,WFC,-0.262413
30128,BAC,NEM,-0.254504
168224,NEM,BAC,-0.254504
79378,ED,CFG,-0.246683
47122,CFG,ED,-0.246683
47315,CFG,O,-0.245414
176843,O,CFG,-0.245414
29952,BAC,ED,-0.242985
79344,ED,BAC,-0.242985


In [101]:
threeday_pairwise[(threeday_pairwise['corr']<-0.20)].sort_values('corr', ascending=True)

,1,2,corr
47298,CFG,NEM,-0.258979
168258,NEM,CFG,-0.258979
168647,NEM,WFC,-0.221833
243743,WFC,NEM,-0.221833
134663,KEY,NEM,-0.201352
168431,NEM,KEY,-0.201352


Other than the different classes of shares for the same firm, which unsurprisingly move closely with each other, energy companies tended to have high 3-day and 5-day correlations over the past 600 trading days.

NEM (Newmont Mining Corporation) looks to have had the strongest opposing movements against bank stocks. Looking at it's performance since 2008, it benefitted greatly from the 2008-2009 crisis, peaking in 2011, and suffered when other companies recovered.

Looking at intra-day movements would of course be very useful for statistical arbitrage purposes. It would also be useful to graph the periodic correlations in addition to taking the average of the entire range.

### Financial Ratios and Performance
Since we have all of the most recent financials, we can look at the relationships between certain ratios and performance in the following period, and potentially identify KPIs for stock performance.

For example, let's see if something relatively boring like the relative inverse of the total debt ratio (total assets/total liabilities) has a relationship with stock performance over the next 10 trading days (or up to the most recent close date). This is a very simplified analysis that does not distinguish between different time periods or even different companies; it just looks at the relationship between the ratio and 30 day stock performance. 

Since there is a delay between the end of a fiscal quarter and the release date, this is not an attempt to measure the direct effects of a financial report on stock performance. But since the current ratio is an indicator of the actual health of a firm, for which people have beliefs about, this is actually more of an attempt to see if market beliefs line up with an unknown reality (excepting potential insider trading...). By limiting the measure to 10 trading days, we're specifically looking at the period before the report has been released.

To get the effects of an earnings report, it would be necessary to find the release dates, which are not provided here.

We define relative inverse total deby ratio as $\frac{\frac{total assets}{total liabilities}}{mean(\frac{total assets}{total liabilities})}$

In [298]:
with open('allfinancial.json') as fin_data:
    fin_dict = json.load(fin_data)

df = []
for ticker in tickers:
    for k,v in fin_dict[ticker]['financials']['quarterly_balance'].items():
        itdr = fin_dict[ticker]['financials']['quarterly_balance'][k]['totalAssets']/fin_dict[ticker]['financials']['quarterly_balance'][k]['totalLiab']
        # Some quarters end on a Sunday, so just add a day if that's the case.
        try:
            performance_10 = close_prices[ticker].iloc[close_prices.index.get_loc(k)+10]/close_prices[ticker].loc[k]-1
        except:
            d = (datetime.datetime(*[int(d) for d in '2017-07-30'.split('-')])+datetime.timedelta(days=1)).strftime("%Y-%m-%d")
            performance_10 = close_prices[ticker].iloc[close_prices.index.get_loc(d)+10]/close_prices[ticker].loc[d]-1
        df.append([performance_10,itdr])
df = pd.DataFrame(df)
df.columns = ['10 Day Performance','Inverse Total Debt Ratio']

df.dropna(inplace=True)

import statsmodels.api as sm
Y = df['10 Day Performance']
X = sm.add_constant(df['Inverse Total Debt Ratio']/np.mean(df['Inverse Total Debt Ratio']))
lm = sm.OLS(Y,X)
results = lm.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     10 Day Performance   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     9.173
Date:                Mon, 18 Dec 2017   Prob (F-statistic):            0.00249
Time:                        15:51:29   Log-Likelihood:                 3492.7
No. Observations:                1913   AIC:                            -6981.
Df Residuals:                    1911   BIC:                            -6970.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                        0.0038      0.002      2.206      0.027       0.000       0.007
Inverse Total Debt Ratio     0.0044      0.001      3.029      0.002       0.002       0.007
==============================================================================
Omnibus:                      343.708   Durbin-Watson:                   1.936
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4466.132
Skew:                           0.435   Prob(JB):                         0.00
Kurtosis:                      10.435   Cond. No.                         3.63
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

So the inverse of the total debt ratio seems to have a statistically significant relationship with stock performance, but by itself it is not a good predictor. Now let's look at the 90-day performance, so that both the before-report and after-report periods are included.

In [299]:
df_90 = []
for ticker in tickers:
    for k,v in fin_dict[ticker]['financials']['quarterly_balance'].items():
        itdr = fin_dict[ticker]['financials']['quarterly_balance'][k]['totalAssets']/fin_dict[ticker]['financials']['quarterly_balance'][k]['totalLiab']
        # Some quarters end on a Sunday, so just add a day if that's the case.
        try:
            performance_90 = close_prices[ticker].iloc[close_prices.index.get_loc(k)+90]/close_prices[ticker].loc[k]-1
        except:
            d = (datetime.datetime(*[int(d) for d in '2017-07-30'.split('-')])+datetime.timedelta(days=1)).strftime("%Y-%m-%d")
            performance_90 = close_prices[ticker].iloc[close_prices.index.get_loc(d)+90]/close_prices[ticker].loc[d]-1
        df_90.append([performance_90,itdr])
df_90 = pd.DataFrame(df_90)
df_90.columns = ['90 Day Performance','Inverse Total Debt Ratio']

df_90.dropna(inplace=True)

Y = df_90['90 Day Performance']
X = sm.add_constant(df_90['Inverse Total Debt Ratio']/np.mean(df_90['Inverse Total Debt Ratio']))
lm = sm.OLS(Y,X)
results = lm.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     90 Day Performance   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     8.201
Date:                Mon, 18 Dec 2017   Prob (F-statistic):            0.00423
Time:                        15:51:37   Log-Likelihood:                 1105.4
No. Observations:                1935   AIC:                            -2207.
Df Residuals:                    1933   BIC:                            -2196.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                        0.0452      0.006      7.535      0.000       0.033       0.057
Inverse Total Debt Ratio     0.0147      0.005      2.864      0.004       0.005       0.025
==============================================================================
Omnibus:                       66.458   Durbin-Watson:                   1.174
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              168.206
Skew:                          -0.113   Prob(JB):                     2.98e-37
Kurtosis:                       4.427   Cond. No.                         3.63
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

These results should not be that surprising, as we might expect that a lower debt ratio/higher inverse ratio would have a larger coefficient (effect) in the longer term, even if by itself it is still not a good predictor. 90 trading days will actually go beyond the next end of quarter.

Obviously, there are more and better ways to use this data. Even sticking with the debt ratio, we might look at how it affects performance within an industry (or every single company individually) and give some recency weighting to ratios. Looking at trends would also be valuable, and examining cash flow and income statements would be helpful as well. Overall, there are many, many more ways in which to use this information.

### Headline/Summary Analysis
Human decisions are based on beliefs. In the stock market context, we have beliefs over the direction of a stock, based on beliefs about the health and direction of the company the stock represents. Since we do not have perfect information over the present (to say nothing of the future), these beliefs can have various levels of truth to them. Over time, more information about the past can be uncovered, and the more information a belief is based on, the more likely it is to be true or rational.

In the short term, the slant of a headline might influence our purchasing decisions. We don't know the precise details of a company's operations, and so we may base our decisions on positive or negative news stories. In the longer term (which in this case may just be the time between two quarterly reports), we will have access to more information, and our beliefs will be either vindicated or proven wrong. However, regardless of how much information is available, sentiment can always be skewed and it is entirely possible for a company with solid fundamentals and growth to be reviled, and vice versa.

This analysis will focus on the shorter term effects for now.

Reddit, Twitter, and other social media are also good sources for sentiment data.

In [390]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, word_tokenize, sent_tokenize

Let's see if headlines from Tuesday, December 5 had some relationship with stock performance for the rest of the week. Or at least the average correlation for every company with it's own Yahoo Finance headlines.

In [301]:
# e.g. Apple's headlines for that day
headline_data['AAPL']['2017-12-05']

{'2 Things Apple Does Wrong': {'publisher': 'Motley Fool',
  'summary': "Though Apple is a great company, it's not perfect.",
  'url': 'https://finance.yahoo.com/news/2-things-apple-does-wrong-184600199.html'},
 '3 Growth Stocks for in-the-Know Investors': {'publisher': 'Motley Fool',
  'summary': "If you don't know, now you know.",
  'url': 'https://finance.yahoo.com/news/3-growth-stocks-know-investors-175200794.html'},
 'Apple Ordered to Pay $15 Billion in Back Taxes to Ireland': {'publisher': 'TheStreet.com',
  'summary': 'Ireland disagreed with the ruling and will file an appeal.',
  'url': 'https://www.thestreet.com/story/14411411/1/apple-ordered-to-pay-back-taxes-to-ireland.html?puc=yahoo&cm_ven=YAHOO&yptr=yahoo'},
 'Apple Remains A Strong Buy As iPad, iMac Demand Picks Up, Says Analyst': {'publisher': 'Benzinga',
  'summary': 'For Apple Inc. (NASDAQ: AAPL ) investors, the name of the game in the closing weeks of 2017 is figuring out just how much demand there is for the iPhone X

In [308]:
# Total number of headlines. Some companies will have a lot more than others, and there will be overlaps.
total_headlines=0
for k,v in headline_data.items():
    try:
        total_headlines += headline_data[k]['2017-12-05']['daily_headlines']
    except:
        pass
total_headlines

938

In [394]:
# Let's combine all headlines and summaries for all stocks into a single dictionary entry.
text_dict = {}
for k,v in headline_data.items():
    text = ''
    for k1,v1 in v.items():
        if k1 == '2017-12-05':
            for k2,v2 in v1.items():
                if k2 != 'daily_headlines':
                    text += ' '+k2+' '+headline_data[k]['2017-12-05'][k2]['summary'] + ' '
    text_dict[k] = text

In [395]:
lines = sent_tokenize(text_dict['AAPL'])
lines

[' Apple buys podcast search startup Pop Up Archive Apple’s played a central role in podcast since its earliest days — heck, even the term was borrowed from its big hardware product at the time.',
 "2 Things Apple Does Wrong Though Apple is a great company, it's not perfect.",
 "Here's how to use Apple Pay Cash, Apple's new Venmo competitor that just launched Apple activated Apple Pay Case this week, a new feature in iOS 11.2.",
 "Here's how to use it.",
 '[$$] Getting Real About Augmented Reality It had all the makings of a long overdue coming-out party for augmented reality.',
 'In June, Apple (AAPL) unfurled a new technology, called ARKit, for its army of developers to create AR apps for hundreds of millions of Apple’s mobile devices and help make the company’s operating system “the largest AR platform in the world overnight,” Apple CEO Tim Cook proclaimed.',
 'The break-out success of Pokemon Go – one of the most-downloaded apps worldwide in 2016 -- and Apple’s promising play under

In [408]:
sid = SIA()
score_list = []
for line in lines:
    print(line)
    scores = sid.polarity_scores(line)
    current_scores = []
    for s in sorted(scores):
        print('{0}: {1}, '.format(s, scores[s]), end='\n')
        current_scores.append(scores[s])
    score_list.append(current_scores)
score_list = pd.DataFrame(score_list)
np.mean(score_list,axis=0)

 Apple buys podcast search startup Pop Up Archive Apple’s played a central role in podcast since its earliest days — heck, even the term was borrowed from its big hardware product at the time.
compound: 0.34, 
neg: 0.0, 
neu: 0.928, 
pos: 0.072, 
2 Things Apple Does Wrong Though Apple is a great company, it's not perfect.
compound: -0.2495, 
neg: 0.318, 
neu: 0.469, 
pos: 0.214, 
Here's how to use Apple Pay Cash, Apple's new Venmo competitor that just launched Apple activated Apple Pay Case this week, a new feature in iOS 11.2.
compound: -0.0772, 
neg: 0.103, 
neu: 0.842, 
pos: 0.055, 
Here's how to use it.
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
[$$] Getting Real About Augmented Reality It had all the makings of a long overdue coming-out party for augmented reality.
compound: 0.4019, 
neg: 0.0, 
neu: 0.87, 
pos: 0.13, 
In June, Apple (AAPL) unfurled a new technology, called ARKit, for its army of developers to create AR apps for hundreds of millions of Apple’s mobile devices 

0    0.175769
1    0.041692
2    0.885885
3    0.072462
dtype: float64

The "compound" score of 0.176 suggests a slightly positive bent, and "neutral" has the highest average score of 0.886. Should we expect too much change over the next few days?

In [398]:
close_prices['AAPL'].loc['2017-12-05':'2017-12-08']

Date
2017-12-05    169.640
2017-12-06    169.010
2017-12-07    169.452
2017-12-08    169.370
Name: AAPL, dtype: float64

Not really, it's down slightly, significantly less than 1%.

With more text from more days, it would not be difficult to look at these relationships over a longer period of time. Other considerations and ideas include:

Different reactions to negative/positive news<br>
Sources of headlines/summaries other than Yahoo Finance (e.g. Bloomberg has an API)<br>
Analysing entire articles<br>
Analysing audio/video related to company (this would be significantly more difficult and not clear if it adds enough relevant info)